# _Development: Nov. 14, 2019_

**_Today's Focus_**

1. Rewrite function that retrieves and stores tweets so that it first retrieves Tweets and places them into a separate dictionary.
    - the entries within this separate dictionary can then be passed to a database table
2. If tweet is not already in table then insert, else don't
3. Grab tweets from `EarnyBot` and store them in database table
    - Create a new tweet from `EarnyBot` and then redo process
    - if it works properly, only one new entry will be added (i.e. the new tweet) as the most recent (i.e. top) entry in the table.  

In [1]:
import dataset
import logging
from joetools import private
import os
import tweepy
import pandas as pd

In [2]:
logger = logging.getLogger()

In [3]:
# set environment variables
private.environment_variables()

In [4]:
def database_connect():
    '''connects to PostgreSQL database and table (via user input)'''
    print('Please insert table name you would like to reference.')
    table_name = str(input())
    
    try:
        # connect to PostgreSQL database
        db = dataset.connect(os.environ.get('CONNECTION_STRING'))
        # create reference to specified table
        table = db.create_table(table_name, primary_id='tweet_id', primary_type=db.types.text)
        return table
    except Exception as e:
        print(e)
        
def api_setup():
    '''set up Twitter API'''

    # access tweepy authenticator
    auth = tweepy.OAuthHandler(os.environ.get('TWITTER_API_KEY'), os.environ.get('TWITTER_API_SECRET'))
    auth.set_access_token(os.environ.get('TWITTER_TOKEN'), os.environ.get('TWITTER_TOKEN_SECRET'))

    # pass in above authentication to API
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
    except Exception as e:
        logger.error('Error creating API', exc_info=True)
        raise e
    logger.info('API created')
    return api

In [ ]:
table = database_connect()

In [ ]:
pd.DataFrame(table.all())

In [ ]:
#table.drop()

In [ ]:
api = api_setup()

In [ ]:
username = str(input())

In [ ]:
def gather_tweets(api, username, num_tweets, table):
    '''function that retrives username and specified number of tweets'''
    # insert database table reference
    table = table
    
    for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(num_tweets):
        # retrieve tweet_id from tweet
        tweet_id = str(tweet.id_str)
        if tweet_id in table:
            print('Tweet ID in table')
            pass
        else:
            created_at = tweet.created_at
            source = tweet.source
            retweet_count = tweet.retweet_count
            favorite_count = tweet.favorite_count
            if tweet.entities['hashtags']:
                hashtags = [n['text'] for n in tweet.entities['hashtags']]
            else:
                hashtags = 'None' 
            if tweet.entities['urls']:
                urls = [n['url'] for n in tweet.entities['urls']]
            else:
                urls = 'None'    
            text = tweet.full_text
            print('Tweet ID not in table and information was successfully retrieved.')

In [ ]:
gather_tweets(api, username, 20, table)

In [ ]:
def gather_tweets(api, username, num_tweets, table):
    '''function that retrives username and specified number of tweets'''
    # insert database table reference
    table = table
    
    for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(num_tweets):
        # retrieve tweet_id from tweet
        tweet_id = str(tweet.id_str)
        if tweet_id in table:
            print('Tweet ID in table, nothing entered.')
            pass
        else:
            created_at = tweet.created_at
            source = tweet.source
            retweet_count = tweet.retweet_count
            favorite_count = tweet.favorite_count
            if tweet.entities['hashtags']:
                hashtags = [n['text'] for n in tweet.entities['hashtags']]
            else:
                hashtags = 'None' 
            if tweet.entities['urls']:
                urls = [n['url'] for n in tweet.entities['urls']]
            else:
                urls = 'None'    
            text = tweet.full_text
            #print('Tweet ID not in table and information was successfully retrieved.')

            # try to insert
            try:
                table.upsert(dict(
                    tweet_id=tweet_id,
                    created_at=created_at,
                    source=source,
                    retweet_count=retweet_count,
                    favorite_count=favorite_count,
                    hashtags=hashtags,
                    urls=urls,
                    text=text), keys=['tweet_id'])
                print('Entry inserted in table.')
            except Exception as e:
                print(e)

In [ ]:
gather_tweets(api, username, 20, table)

In [ ]:
import pandas as pd

pd.DataFrame(table.all()).head()

In [ ]:
pd.DataFrame(table.all()).tail()

In [ ]:
pd.DataFrame(table.all())

**ENTERED IN THE FOLLOWING TWEET: When the winds of change blow, some people build walls and others build windmills. -- Chinese proverb**

Now we will check to see if the database gets correctly updated after tweeting the above text.

In [ ]:
gather_tweets(api, username, 20, table)

In [ ]:
pd.DataFrame(table.all())

**ENTERED IN THE FOLLOWING TWEETS:**
- "The dogmas of the quiet past, are inadequate to the stormy present. The occasion is piled high with difficulty, and we must rise with the occasion. As our case is new, so we must think anew and act anew." -- Abraham Lincoln
- “Every man has two lives, and the second begins when we realize we only have one.” -- Confucius

Now we will check to see if the database gets correctly updated after tweeting the above text.

In [ ]:
gather_tweets(api, username, 50, table)

In [ ]:
pd.DataFrame(table.all())

In [ ]:
table.drop()

# _RESTART_

In [ ]:
table = database_connect()

In [ ]:
pd.DataFrame(table.all())

In [ ]:
api = api_setup()

In [ ]:
username = str(input())

In [19]:
def gather_tweets(api, username, num_tweets, table):
    '''function that retrives username and specified number of tweets'''
    # insert database table reference
    table = table
    
    for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(num_tweets):
        # retrieve tweet_id from tweet
        tweet_id = str(tweet.id_str)
        if tweet_id in table:
            print('Tweet ID in table, nothing entered.')
            return None
        else:
            created_at = tweet.created_at
            source = tweet.source
            retweet_count = tweet.retweet_count
            favorite_count = tweet.favorite_count
            if tweet.entities['hashtags']:
                hashtags = [n['text'] for n in tweet.entities['hashtags']]
            else:
                hashtags = 'None' 
            if tweet.entities['urls']:
                urls = [n['url'] for n in tweet.entities['urls']]
            else:
                urls = 'None'    
            text = tweet.full_text
            #print('Tweet ID not in table and information was successfully retrieved.')

            # try to insert
            try:
                table.upsert(dict(
                    tweet_id=tweet_id,
                    created_at=created_at,
                    source=source,
                    retweet_count=retweet_count,
                    favorite_count=favorite_count,
                    hashtags=hashtags,
                    urls=urls,
                    text=text), keys=['tweet_id'])
                print('Entry from {} inserted in table.'.format(str(created_at)))
            except Exception as e:
                print(e)

In [ ]:
gather_tweets(api, username, 50, table)

In [ ]:
gather_tweets(api, username, 50, table)

In [ ]:
gather_tweets(api, username, 50, table)

In [ ]:
gather_tweets(api, username, 50, table)

In [ ]:
df = pd.DataFrame(table.all(), index=[''])

In [ ]:
df.sort_values('created_at', ascending=False).set_index('created_at')

# _Restart #2_

- Use `created_at` as key and see if it stores in chronological order

In [6]:
# changed primary_id to created_at and the type to datetime
def database_connect():
    '''connects to PostgreSQL database and table (via user input)'''
    print('Please insert table name you would like to reference.')
    table_name = str(input())
    
    try:
        # connect to PostgreSQL database
        db = dataset.connect(os.environ.get('CONNECTION_STRING'))
        # create reference to specified table
        table = db.create_table(table_name, primary_id='created_at', primary_type=db.types.datetime)
        return table
    except Exception as e:
        print(e)

In [7]:
#table.drop()

In [8]:
table = database_connect()

Please insert table name you would like to reference.


 earnybot_nov14


In [9]:
pd.DataFrame(table.all())

,tweet_id,created_at,source,retweet_count,favorite_count,hashtags,urls,text
0,1194007043230842880,2019-11-11 21:40:37,EarnyBot,0,0,None,None,"""How ridiculous and how strange to be surprise..."
1,1193745939795763201,2019-11-11 04:23:05,Twitter Web App,1,0,"{tweepy,python,DataScience}",{https://t.co/yy9thIWR4P},RT @earny_joe: A few code examples for working...
2,1193319058227236865,2019-11-10 00:06:48,EarnyBot,116,0,None,None,"RT @NavalBot: ""Develop 'strategic incompetence..."
3,1193319049054343169,2019-11-10 00:06:46,EarnyBot,226,0,None,None,"RT @NavalBot: ""When it comes to medicine and n..."
4,1193318312471605249,2019-11-10 00:03:51,EarnyBot,858,0,None,None,"RT @NavalBot: ""The fundamental delusion - ther..."
5,1193304066291580931,2019-11-09 23:07:14,EarnyBot,0,0,None,None,"""Our life is what our thoughts make it."" -- Ma..."
6,1193304041100644353,2019-11-09 23:07:08,EarnyBot,0,0,None,None,"""You can commit injustice by doing nothing."" -..."
7,1193295800438317056,2019-11-09 22:34:23,EarnyBot,0,0,None,None,"""Today I escaped anxiety. Or no, I discarded i..."
8,1193295775175970816,2019-11-09 22:34:17,EarnyBot,0,0,None,None,"""The best answer to anger is silence."" -- Marc..."
9,1195065110676594688,2019-11-14 19:45:00,Twitter Web App,0,0,None,None,"""Taking a new step, uttering a new word, is wh..."


In [10]:
table.drop()

In [11]:
pd.DataFrame(table.all())

""


In [12]:
table = database_connect()

Please insert table name you would like to reference.


 earnybot_nov14


In [13]:
api = api_setup()

In [14]:
username = str(input())

 EarnyBot


In [15]:
gather_tweets(api, username, 50, table)

Entry from 2019-11-14 19:45:00 inserted in table.
Entry from 2019-11-14 19:42:56 inserted in table.
Entry from 2019-11-14 19:31:38 inserted in table.
Entry from 2019-11-14 19:28:13 inserted in table.
Entry from 2019-11-14 19:25:25 inserted in table.
Entry from 2019-11-14 19:07:11 inserted in table.
Entry from 2019-11-14 04:28:25 inserted in table.
Entry from 2019-11-13 23:52:29 inserted in table.
Entry from 2019-11-13 23:46:34 inserted in table.
Entry from 2019-11-11 21:43:59 inserted in table.
Entry from 2019-11-11 21:40:54 inserted in table.
Entry from 2019-11-11 21:40:45 inserted in table.
Entry from 2019-11-11 21:40:37 inserted in table.
Entry from 2019-11-11 04:23:05 inserted in table.
Entry from 2019-11-10 00:06:48 inserted in table.
Entry from 2019-11-10 00:06:46 inserted in table.
Entry from 2019-11-10 00:03:51 inserted in table.
Entry from 2019-11-09 23:07:14 inserted in table.
Entry from 2019-11-09 23:07:08 inserted in table.
Entry from 2019-11-09 22:34:23 inserted in table.


In [22]:
pd.DataFrame(table.all())

,created_at,tweet_id,source,retweet_count,favorite_count,hashtags,urls,text
0,2019-11-14 21:14:31,1195087638665019392,Twitter Web App,0,0,None,None,"""If you want to improve, you must be content t..."
1,2019-11-14 19:45:00,1195065110676594688,Twitter Web App,0,0,None,None,"""Taking a new step, uttering a new word, is wh..."
2,2019-11-14 19:42:56,1195064590087806976,Twitter Web App,0,0,None,None,"""All men make mistakes, but a good man yields ..."
3,2019-11-14 19:31:38,1195061749625556992,Twitter Web App,0,0,None,None,"“Every man has two lives, and the second begin..."
4,2019-11-14 19:28:13,1195060887985557504,Twitter Web App,0,0,None,None,"""The dogmas of the quiet past, are inadequate ..."
5,2019-11-14 19:25:25,1195060182369394688,Twitter Web App,0,0,None,None,"""When the winds of change blow, some people bu..."
6,2019-11-14 19:07:11,1195055596032143360,Twitter Web App,0,0,None,None,"""Let him that would move the world first move ..."
7,2019-11-14 04:28:25,1194834444512845824,Twitter Web App,0,0,None,None,"""Out on the edge you see all kinds of things y..."
8,2019-11-13 23:52:29,1194765004647673857,Twitter Web App,0,0,None,None,"""Understand: the future belongs to groups that..."
9,2019-11-13 23:46:34,1194763514105286658,Twitter Web App,0,0,None,None,"""The best fighter is never angry."" -- Lao Tzu"


In [20]:
gather_tweets(api, username, 50, table)

Entry from 2019-11-14 21:14:31 inserted in table.
Entry from 2019-11-14 19:45:00 inserted in table.
Entry from 2019-11-14 19:42:56 inserted in table.
Entry from 2019-11-14 19:31:38 inserted in table.
Entry from 2019-11-14 19:28:13 inserted in table.
Entry from 2019-11-14 19:25:25 inserted in table.
Entry from 2019-11-14 19:07:11 inserted in table.
Entry from 2019-11-14 04:28:25 inserted in table.
Entry from 2019-11-13 23:52:29 inserted in table.
Entry from 2019-11-13 23:46:34 inserted in table.
Entry from 2019-11-11 21:43:59 inserted in table.
Entry from 2019-11-11 21:40:54 inserted in table.
Entry from 2019-11-11 21:40:45 inserted in table.
Entry from 2019-11-11 21:40:37 inserted in table.
Entry from 2019-11-11 04:23:05 inserted in table.
Entry from 2019-11-10 00:06:48 inserted in table.
Entry from 2019-11-10 00:06:46 inserted in table.
Entry from 2019-11-10 00:03:51 inserted in table.
Entry from 2019-11-09 23:07:14 inserted in table.
Entry from 2019-11-09 23:07:08 inserted in table.


In [23]:
pd.DataFrame(table.all())

,created_at,tweet_id,source,retweet_count,favorite_count,hashtags,urls,text
0,2019-11-14 21:14:31,1195087638665019392,Twitter Web App,0,0,None,None,"""If you want to improve, you must be content t..."
1,2019-11-14 19:45:00,1195065110676594688,Twitter Web App,0,0,None,None,"""Taking a new step, uttering a new word, is wh..."
2,2019-11-14 19:42:56,1195064590087806976,Twitter Web App,0,0,None,None,"""All men make mistakes, but a good man yields ..."
3,2019-11-14 19:31:38,1195061749625556992,Twitter Web App,0,0,None,None,"“Every man has two lives, and the second begin..."
4,2019-11-14 19:28:13,1195060887985557504,Twitter Web App,0,0,None,None,"""The dogmas of the quiet past, are inadequate ..."
5,2019-11-14 19:25:25,1195060182369394688,Twitter Web App,0,0,None,None,"""When the winds of change blow, some people bu..."
6,2019-11-14 19:07:11,1195055596032143360,Twitter Web App,0,0,None,None,"""Let him that would move the world first move ..."
7,2019-11-14 04:28:25,1194834444512845824,Twitter Web App,0,0,None,None,"""Out on the edge you see all kinds of things y..."
8,2019-11-13 23:52:29,1194765004647673857,Twitter Web App,0,0,None,None,"""Understand: the future belongs to groups that..."
9,2019-11-13 23:46:34,1194763514105286658,Twitter Web App,0,0,None,None,"""The best fighter is never angry."" -- Lao Tzu"


In [24]:
gather_tweets(api, username, 50, table)

Entry from 2019-11-14 21:36:04 inserted in table.
Entry from 2019-11-14 21:14:31 inserted in table.
Entry from 2019-11-14 19:45:00 inserted in table.
Entry from 2019-11-14 19:42:56 inserted in table.
Entry from 2019-11-14 19:31:38 inserted in table.
Entry from 2019-11-14 19:28:13 inserted in table.
Entry from 2019-11-14 19:25:25 inserted in table.
Entry from 2019-11-14 19:07:11 inserted in table.
Entry from 2019-11-14 04:28:25 inserted in table.
Entry from 2019-11-13 23:52:29 inserted in table.
Entry from 2019-11-13 23:46:34 inserted in table.
Entry from 2019-11-11 21:43:59 inserted in table.
Entry from 2019-11-11 21:40:54 inserted in table.
Entry from 2019-11-11 21:40:45 inserted in table.
Entry from 2019-11-11 21:40:37 inserted in table.
Entry from 2019-11-11 04:23:05 inserted in table.
Entry from 2019-11-10 00:06:48 inserted in table.
Entry from 2019-11-10 00:06:46 inserted in table.
Entry from 2019-11-10 00:03:51 inserted in table.
Entry from 2019-11-09 23:07:14 inserted in table.


In [25]:
pd.DataFrame(table.all())

,created_at,tweet_id,source,retweet_count,favorite_count,hashtags,urls,text
0,2019-11-13 23:52:29,1194765004647673857,Twitter Web App,0,0,None,None,"""Understand: the future belongs to groups that..."
1,2019-11-13 23:46:34,1194763514105286658,Twitter Web App,0,0,None,None,"""The best fighter is never angry."" -- Lao Tzu"
2,2019-11-11 21:43:59,1194007890757083141,EarnyBot,0,0,None,None,"""Do not fear mistakes. You will know failure. ..."
3,2019-11-11 21:40:54,1194007113628110848,EarnyBot,0,0,None,None,"""An investment in knowledge pays the best inte..."
4,2019-11-11 21:40:45,1194007078349750272,EarnyBot,0,0,None,None,"""To be yourself in a world that is constantly ..."
5,2019-11-11 21:40:37,1194007043230842880,EarnyBot,0,0,None,None,"""How ridiculous and how strange to be surprise..."
6,2019-11-11 04:23:05,1193745939795763201,Twitter Web App,1,0,"{tweepy,python,DataScience}",{https://t.co/yy9thIWR4P},RT @earny_joe: A few code examples for working...
7,2019-11-14 21:36:04,1195093063372288000,Twitter Web App,0,0,None,None,"""My opponent is my teacher, my ego is my enemy..."
8,2019-11-14 21:14:31,1195087638665019392,Twitter Web App,0,0,None,None,"""If you want to improve, you must be content t..."
9,2019-11-14 19:45:00,1195065110676594688,Twitter Web App,0,0,None,None,"""Taking a new step, uttering a new word, is wh..."


In [28]:
pd.DataFrame(table.find(order_by=['-created_at'])).set_index('created_at')

,tweet_id,source,retweet_count,favorite_count,hashtags,urls,text
created_at,,,,,,,
2019-11-14 21:36:04,1195093063372288000,Twitter Web App,0,0,None,None,"""My opponent is my teacher, my ego is my enemy..."
2019-11-14 21:14:31,1195087638665019392,Twitter Web App,0,0,None,None,"""If you want to improve, you must be content t..."
2019-11-14 19:45:00,1195065110676594688,Twitter Web App,0,0,None,None,"""Taking a new step, uttering a new word, is wh..."
2019-11-14 19:42:56,1195064590087806976,Twitter Web App,0,0,None,None,"""All men make mistakes, but a good man yields ..."
2019-11-14 19:31:38,1195061749625556992,Twitter Web App,0,0,None,None,"“Every man has two lives, and the second begin..."
2019-11-14 19:28:13,1195060887985557504,Twitter Web App,0,0,None,None,"""The dogmas of the quiet past, are inadequate ..."
2019-11-14 19:25:25,1195060182369394688,Twitter Web App,0,0,None,None,"""When the winds of change blow, some people bu..."
2019-11-14 19:07:11,1195055596032143360,Twitter Web App,0,0,None,None,"""Let him that would move the world first move ..."
2019-11-14 04:28:25,1194834444512845824,Twitter Web App,0,0,None,None,"""Out on the edge you see all kinds of things y..."


# _RESTART #3_

In [15]:
#table = database_connect()
#pd.DataFrame(table.all())
#table.drop()
#pd.DataFrame(table.all())
#table = database_connect()
#api = api_setup()
#username = str(input())

In [4]:
# changed primary_id to created_at and the type to datetime
def database_connect():
    '''connects to PostgreSQL database and table (via user input)'''
    print('Please insert table name you would like to reference.')
    table_name = str(input())
    
    try:
        # connect to PostgreSQL database
        db = dataset.connect(os.environ.get('CONNECTION_STRING'))
        # create reference to specified table
        table = db.create_table(table_name, primary_id='tweet_id', primary_type=db.types.text)
        return table
    except Exception as e:
        print(e)

In [19]:
def gather_tweets(api, username, num_tweets, table):
    '''function that retrives username and specified number of tweets'''
    # insert database table reference
    table = table
    
    for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(num_tweets):
        # retrieve tweet_id from tweet
        tweet_id = tweet.id_str
        if table.find_one(tweet_id=tweet_id) is not None:
            print('Tweet ID {} in table, entry passed.'.format(tweet_id))
        else:
            created_at = tweet.created_at
            source = tweet.source
            retweet_count = tweet.retweet_count
            favorite_count = tweet.favorite_count
            if tweet.entities['hashtags']:
                hashtags = [n['text'] for n in tweet.entities['hashtags']]
            else:
                hashtags = 'None' 
            if tweet.entities['urls']:
                urls = [n['url'] for n in tweet.entities['urls']]
            else:
                urls = 'None'    
            text = tweet.full_text
            print('Tweet ID not in table and information was successfully retrieved.')

            # try to insert
            try:
                table.upsert(dict(
                    tweet_id=tweet_id,
                    created_at=created_at,
                    source=source,
                    retweet_count=retweet_count,
                    favorite_count=favorite_count,
                    hashtags=hashtags,
                    urls=urls,
                    text=text), keys=['tweet_id'])
                print('Entry from {} inserted in table.'.format(str(created_at)))
            except Exception as e:
                print(e)

In [6]:
table = database_connect()

Please insert table name you would like to reference.


 earnybot_nov14


In [7]:
pd.DataFrame(table.all())

,created_at,tweet_id,source,retweet_count,favorite_count,hashtags,urls,text
0,2019-11-13 23:52:29,1194765004647673857,Twitter Web App,0,0,None,None,"""Understand: the future belongs to groups that..."
1,2019-11-13 23:46:34,1194763514105286658,Twitter Web App,0,0,None,None,"""The best fighter is never angry."" -- Lao Tzu"
2,2019-11-11 21:43:59,1194007890757083141,EarnyBot,0,0,None,None,"""Do not fear mistakes. You will know failure. ..."
3,2019-11-11 21:40:54,1194007113628110848,EarnyBot,0,0,None,None,"""An investment in knowledge pays the best inte..."
4,2019-11-11 21:40:45,1194007078349750272,EarnyBot,0,0,None,None,"""To be yourself in a world that is constantly ..."
5,2019-11-11 21:40:37,1194007043230842880,EarnyBot,0,0,None,None,"""How ridiculous and how strange to be surprise..."
6,2019-11-11 04:23:05,1193745939795763201,Twitter Web App,1,0,"{tweepy,python,DataScience}",{https://t.co/yy9thIWR4P},RT @earny_joe: A few code examples for working...
7,2019-11-14 21:36:04,1195093063372288000,Twitter Web App,0,0,None,None,"""My opponent is my teacher, my ego is my enemy..."
8,2019-11-14 21:14:31,1195087638665019392,Twitter Web App,0,0,None,None,"""If you want to improve, you must be content t..."
9,2019-11-14 19:45:00,1195065110676594688,Twitter Web App,0,0,None,None,"""Taking a new step, uttering a new word, is wh..."


In [8]:
table.drop()

In [9]:
pd.DataFrame(table.all())

""


In [10]:
table = database_connect()

Please insert table name you would like to reference.


 earnybot_nov14


In [12]:
def api_setup():
    '''set up Twitter API'''

    # access tweepy authenticator
    auth = tweepy.OAuthHandler(os.environ.get('TWITTER_API_KEY'), os.environ.get('TWITTER_API_SECRET'))
    auth.set_access_token(os.environ.get('TWITTER_TOKEN'), os.environ.get('TWITTER_TOKEN_SECRET'))

    # pass in above authentication to API
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    try:
        api.verify_credentials()
    except Exception as e:
        logger.error('Error creating API', exc_info=True)
        raise e
    logger.info('API created')
    return api

In [13]:
api = api_setup()

In [14]:
username = str(input())

 EarnyBot


In [18]:
gather_tweets(api, username, 50, table)

Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID not in table and information was successfully retrieved.
Tweet ID n

In [20]:
gather_tweets(api, username, 50, table)

Tweet ID not in table and information was successfully retrieved.
Entry from 2019-11-14 21:36:04 inserted in table.
Tweet ID not in table and information was successfully retrieved.
Entry from 2019-11-14 21:14:31 inserted in table.
Tweet ID not in table and information was successfully retrieved.
Entry from 2019-11-14 19:45:00 inserted in table.
Tweet ID not in table and information was successfully retrieved.
Entry from 2019-11-14 19:42:56 inserted in table.
Tweet ID not in table and information was successfully retrieved.
Entry from 2019-11-14 19:31:38 inserted in table.
Tweet ID not in table and information was successfully retrieved.
Entry from 2019-11-14 19:28:13 inserted in table.
Tweet ID not in table and information was successfully retrieved.
Entry from 2019-11-14 19:25:25 inserted in table.
Tweet ID not in table and information was successfully retrieved.
Entry from 2019-11-14 19:07:11 inserted in table.
Tweet ID not in table and information was successfully retrieved.
Entry 

In [21]:
gather_tweets(api, username, 50, table)

Tweet ID 1195093063372288000 in table, entry passed.
Tweet ID 1195087638665019392 in table, entry passed.
Tweet ID 1195065110676594688 in table, entry passed.
Tweet ID 1195064590087806976 in table, entry passed.
Tweet ID 1195061749625556992 in table, entry passed.
Tweet ID 1195060887985557504 in table, entry passed.
Tweet ID 1195060182369394688 in table, entry passed.
Tweet ID 1195055596032143360 in table, entry passed.
Tweet ID 1194834444512845824 in table, entry passed.
Tweet ID 1194765004647673857 in table, entry passed.
Tweet ID 1194763514105286658 in table, entry passed.
Tweet ID 1194007890757083141 in table, entry passed.
Tweet ID 1194007113628110848 in table, entry passed.
Tweet ID 1194007078349750272 in table, entry passed.
Tweet ID 1194007043230842880 in table, entry passed.
Tweet ID 1193745939795763201 in table, entry passed.
Tweet ID 1193319058227236865 in table, entry passed.
Tweet ID 1193319049054343169 in table, entry passed.
Tweet ID 1193318312471605249 in table, entry p

In [23]:
gather_tweets(api, username, 50, table)

Tweet ID not in table and information was successfully retrieved.
Entry from 2019-11-14 21:55:23 inserted in table.
Tweet ID 1195093063372288000 in table, entry passed.
Tweet ID 1195087638665019392 in table, entry passed.
Tweet ID 1195065110676594688 in table, entry passed.
Tweet ID 1195064590087806976 in table, entry passed.
Tweet ID 1195061749625556992 in table, entry passed.
Tweet ID 1195060887985557504 in table, entry passed.
Tweet ID 1195060182369394688 in table, entry passed.
Tweet ID 1195055596032143360 in table, entry passed.
Tweet ID 1194834444512845824 in table, entry passed.
Tweet ID 1194765004647673857 in table, entry passed.
Tweet ID 1194763514105286658 in table, entry passed.
Tweet ID 1194007890757083141 in table, entry passed.
Tweet ID 1194007113628110848 in table, entry passed.
Tweet ID 1194007078349750272 in table, entry passed.
Tweet ID 1194007043230842880 in table, entry passed.
Tweet ID 1193745939795763201 in table, entry passed.
Tweet ID 1193319058227236865 in tabl

In [29]:
for n in table.find(order_by=['-created_at']):
    print(n['created_at'])

2019-11-14 21:55:23
2019-11-14 21:36:04
2019-11-14 21:14:31
2019-11-14 19:45:00
2019-11-14 19:42:56
2019-11-14 19:31:38
2019-11-14 19:28:13
2019-11-14 19:25:25
2019-11-14 19:07:11
2019-11-14 04:28:25
2019-11-13 23:52:29
2019-11-13 23:46:34
2019-11-11 21:43:59
2019-11-11 21:40:54
2019-11-11 21:40:45
2019-11-11 21:40:37
2019-11-11 04:23:05
2019-11-10 00:06:48
2019-11-10 00:06:46
2019-11-10 00:03:51
2019-11-09 23:07:14
2019-11-09 23:07:08
2019-11-09 22:34:23
2019-11-09 22:34:17


In [30]:
def gather_tweets(api, username, num_tweets, table):
    '''function that retrives username and specified number of tweets'''
    # insert database table reference
    table = table
    
    for tweet in tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(num_tweets):
        # retrieve tweet_id from tweet
        tweet_id = tweet.id_str    
        if table.find_one(tweet_id=tweet_id) is not None:
            pass
        else:
            created_at = tweet.created_at
            source = tweet.source
            retweet_count = tweet.retweet_count
            favorite_count = tweet.favorite_count
            if tweet.entities['hashtags']:
                hashtags = [n['text'] for n in tweet.entities['hashtags']]
            else:
                hashtags = 'None' 
            if tweet.entities['urls']:
                urls = [n['url'] for n in tweet.entities['urls']]
            else:
                urls = 'None'    
            text = tweet.full_text
            print('Tweet ID not in table and information was successfully retrieved.')

            # try to insert
            try:
                table.upsert(dict(
                    tweet_id=tweet_id,
                    created_at=created_at,
                    source=source,
                    retweet_count=retweet_count,
                    favorite_count=favorite_count,
                    hashtags=hashtags,
                    urls=urls,
                    text=text), keys=['tweet_id'])
                print('Tweet ID {} from {} inserted in table.'.format(str(created_at)))
            except Exception as e:
                print(e)

In [31]:
gather_tweets(api, username, 50, table)

In [33]:
import time

while time.sleep(5):
    print('True')

In [34]:
def num_tweets():
    while True:
        amount = input("Enter number of tweets you'd like to gather: ")
        try:
            val = int(amount)
            if val >= 0 and val <= 3200:
                break
            else:
                print("Number of tweets must be between 0 and 3,200. Please try again.")
        except ValueError:
            print("Amount must be a number, try again")
    return val

In [35]:
num_tweets()

Enter number of tweets you'd like to gather:  -1


Number of tweets must be between 0 and 3,200. Please try again.


Enter number of tweets you'd like to gather:  3201


Number of tweets must be between 0 and 3,200. Please try again.


Enter number of tweets you'd like to gather:  3200


3200

In [49]:
def get_username():
    while True:
        username = str(input("Enter username of Twitter account: "))
        try:
            if api.get_user(id=username):
                break
        except Exception:
            print('Please enter a valid username.')
    return username

In [50]:
get_username()

Enter username of Twitter account:  alksdjflaksjdhf


Please enter a valid username.


Enter username of Twitter account:  EarnyBot


'EarnyBot'

In [51]:
get_username()

Enter username of Twitter account:  lsdhflasdjhflasjdf


Please enter a valid username.


Enter username of Twitter account:  aksdjhfgkjasdgf


Please enter a valid username.


Enter username of Twitter account:  EarnyBot


'EarnyBot'